In [41]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [48]:
from neo4j import GraphDatabase
from rdflib_neo4j import Neo4jStoreConfig, Neo4jStore, HANDLE_VOCAB_URI_STRATEGY
from rdflib import Graph
import perfumeKGQueries as pkg
from  pprint import pprint

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://53601604.databases.neo4j.io"
AUTH = ("neo4j", "P2Ui4U8o4mpSotvcLNAVXock_1IpcJh5rFANct7uKSc")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [50]:
def retrieve_and_print_all_accords(driver):
    with driver.session() as session:
        try:
            def get_all_accords(tx):
                query = """
                MATCH (a:Accord)
                RETURN a.uri as uri, a.label as name
                """
                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            with driver.session() as session:
                accords = session.read_transaction(get_all_accords)
                print("All Accords:")
                for accord in accords:
                        print(f"name: {accord['name']}, uri: {accord['uri']}")
                        
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_perfumes_with_accord(driver, accordURI):
    with driver.session() as session:
        try:
            def get_perfumes(tx):
                query = pkg.perfumesHaveAccord(accordURI)

                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            with driver.session() as session:
                accords = session.read_transaction(get_perfumes)
                print("All Perfumes with Searched Accord:")
                for accord in accords:
                        print(f"name: {accord['name']}, uri: {accord['uri']}")
                        
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_perfumes_similar_by_accord(driver, perfume_uri):
    with driver.session() as session:
        try:
            def get_similar_perfumes(tx):
                query = pkg.perfumesShareAccords(perfume_uri)

                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['Perfume'], 'score': record['MutualAccords']} for record in result]

            with driver.session() as session:
                accords = session.read_transaction(get_similar_perfumes)
                print("Top 10 Similar Perfumes:")
                for accord in accords:
                        print(f"name: {accord['name']}, uri: {accord['uri']}, score: {accord['score']}")
                        
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_and_print_all_notes(driver):
    with driver.session() as session:
        try:
            def get_all_notes(tx):
                query = """
                MATCH (n:Note)
                RETURN n.uri as uri, n.label as name
                """
                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            notes = session.read_transaction(get_all_notes)
            print("All Notes:")
            for note in notes:
                print(f"name: {note['name']}, uri: {note['uri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_perfumes_with_note(driver, noteURI):
    with driver.session() as session:
        try:
            def get_perfumes(tx):
                query = pkg.perfumesHaveNote(noteURI)

                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            perfumes = session.read_transaction(get_perfumes)
            print("All Perfumes with Searched Note:")
            for perfume in perfumes:
                print(f"name: {perfume['name']}, uri: {perfume['uri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_perfumes_similar_by_note(driver, perfume_uri):
    with driver.session() as session:
        try:
            def get_similar_perfumes(tx):
                query = pkg.perfumesShareNotes(perfume_uri)

                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['Perfume'], 'score': record['MutualNotes']} for record in result]

            similar_perfumes = session.read_transaction(get_similar_perfumes)
            print("Top 10 Similar Perfumes:")
            for perfume in similar_perfumes:
                print(f"name: {perfume['name']}, uri: {perfume['uri']}, score: {perfume['score']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_and_print_all_brands(driver):
    with driver.session() as session:
        try:
            def get_all_brands(tx):
                query = pkg.getAllBrands()
                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            brands = session.read_transaction(get_all_brands)
            print("All Brands:")
            for brand in brands:
                print(f"name: {brand['name']}, uri: {brand['uri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_notes_by_brand(driver, brand_uri):
    with driver.session() as session:
        try:
            def get_notes(tx):
                query = pkg.getNotesUsedByBrand(brand_uri)
                result = tx.run(query)
                notes = []
                for record in result:
                    notes.append({
                        'noteUri': record['noteUri'],
                        'noteName': record['noteName']
                    })
                return notes

            notes = session.read_transaction(get_notes)
            print(f"Notes used by Brand URI: {brand_uri}")
            for note in notes:
                print(f"Note - name: {note['noteName']}, uri: {note['noteUri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_accords_by_brand(driver, brand_uri):
    with driver.session() as session:
        try:
            def get_accords(tx):
                query = pkg.getAccordsUsedByBrand(brand_uri)
                result = tx.run(query)
                accords = []
                for record in result:
                    accords.append({
                        'accordUri': record['accordUri'],
                        'accordName': record['accordName']
                    })
                return accords

            accords = session.read_transaction(get_accords)
            print(f"Accords used by Brand URI: {brand_uri}")
            for accord in accords:
                print(f"Accord - name: {accord['accordName']}, uri: {accord['accordUri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_most_used_accord_for_brand(driver, brand_uri):
    with driver.session() as session:
        try:
            def get_most_used_accord(tx):
                query = pkg.getMostUsedAccordForBrand(brand_uri)
                result = tx.run(query)
                record = result.single()
                if record:
                    return {
                        'brandUri': brand_uri,
                        'accordName': record['accordName'],
                        'accordUri': record['accordUri'],
                        'usageCount': record['usageCount']
                    }
                else:
                    return None

            accord = session.read_transaction(get_most_used_accord)
            if accord:
                print(f"Most Used Accord for Brand URI {brand_uri}:")
                print(f"Accord: {accord['accordName']}, URI: {accord['accordUri']}, Usage Count: {accord['usageCount']}")
            else:
                print(f"No accords found for Brand URI {brand_uri}")
                                
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_accord_similarity_scores(driver, perfume_uri):
    with driver.session() as session:
        try:
            def get_similarity_info(tx):
                query = pkg.getAccordSimilarityScore(perfume_uri)
                result = tx.run(query)
                return [{
                    'Perfume': record['Perfume'],
                    'uri': record['uri'],
                    'similarity': record['similarity']
                } for record in result]

            perfumes_info = session.read_transaction(get_similarity_info)
            return perfumes_info
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_note_similarity_scores(driver, perfume_uri):
    with driver.session() as session:
        try:
            def get_similarity_info(tx):
                query = pkg.getNoteSimilarityScore(perfume_uri)
                result = tx.run(query)
                return [{
                    'Perfume': record['Perfume'],
                    'uri': record['uri'],
                    'similarity': record['similarity']
                } for record in result]

            perfumes_info = session.read_transaction(get_similarity_info)
            return perfumes_info
                                
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_combined_similarity_scores(driver, perfume_uri):
    accord_similarities = retrieve_accord_similarity_scores(driver, perfume_uri)
    note_similarities = retrieve_note_similarity_scores(driver, perfume_uri)
    
    combined_scores = {}

    # Process accord similarities
    if accord_similarities:
        for accord_info in accord_similarities:
            uri = accord_info['uri']
            accord_similarity = accord_info.get('similarity') or 0  # Set to 0 if None
            combined_scores[uri] = {
                'Perfume': accord_info['Perfume'],
                'uri': uri,
                'accordSimilarity': accord_similarity,
                'noteSimilarity': 0  # Default to 0
            }

    # Process note similarities
    if note_similarities:
        for note_info in note_similarities:
            uri = note_info['uri']
            note_similarity = note_info.get('similarity') or 0  # Set to 0 if None
            if uri in combined_scores:
                combined_scores[uri]['noteSimilarity'] = note_similarity
            else:
                combined_scores[uri] = {
                    'Perfume': note_info['Perfume'],
                    'uri': uri,
                    'accordSimilarity': 0,
                    'noteSimilarity': note_similarity
                }

    # Handle perfumes that are not in accord_similarities or note_similarities
    # If both accord_similarities and note_similarities are empty, combined_scores will be empty

    # Calculate averageSimilarity
    for perfume in combined_scores.values():
        perfume['averageSimilarity'] = (perfume['accordSimilarity'] + perfume['noteSimilarity']) / 2.0

    # Sort the perfumes by averageSimilarity in descending order and take top 10
    top_perfumes = sorted(combined_scores.values(), key=lambda x: x['averageSimilarity'], reverse=True)[:10]

    print("Top 10 Combined Similarity Scores:")
    for perfume in top_perfumes:
        print(f"Name: {perfume['Perfume']}")
        print(f"URI: {perfume['uri']}")
        print(f"Accord Similarity: {perfume['accordSimilarity']}")
        print(f"Note Similarity: {perfume['noteSimilarity']}")
        print(f"Average Similarity: {perfume['averageSimilarity']}")
        print("---")

def retrieve_perfumes(driver, count):   
    with driver.session() as session:
        try:
            def get_all_perfumes(tx):
                query = f""" 
                        MATCH (p:Perfume)
                        RETURN p.label as Name, p.uri as URI, p.description as Description
                        LIMIT {count}
                    """
                result = tx.run(query)
                result = [{'URI': record['URI'], 'Name': record['Name'], 'Description': record['Description']} for record in result]   

                return result

            with driver.session() as session:
                perfumes = session.read_transaction(get_all_perfumes)
                print("5 Sample Perfumes:")
                for perfume in perfumes:
                    pprint(f"Name: {perfume['Name']}, URI: {perfume['URI']}, Description: {perfume['Description']}")
                    print("---")
                        
        except Exception as e:
            print(f"An error occurred: {e}")

# List Perfumes

In [52]:
retrieve_perfumes(driver, 500)

5 Sample Perfumes:
('Name: 00 auriel eau de parfum, URI: '
 'http://www.example.org/perfumeKG#Perfume_00-auriel-eau-de-parfum, '
 'Description:  No. 00 Auriel is a dark, sultry, animalic wood that balances '
 'feminine and masculine from top to bottom. At first impression, the scent '
 'opens with an assertive blend of cassis berries, herbaceous coriander and '
 'tellicherry peppercorns. The heart is a copious floral aphrodisiac of spicy '
 'rose absolute, creamy jasmin sambac and sweet muguet giving a smooth velvety '
 "texture. Auriel's roots of black agarwood develop on the skin with resinous "
 'warmth and richness that is undeniably provocative.')
---
('Name: #002, URI: http://www.example.org/perfumeKG#Perfume_002, Description: '
 'None')
---
('Name: 004 gin mandarine musc, URI: '
 'http://www.example.org/perfumeKG#Perfume_004-gin-mandarine-musc, '
 'Description: 004 Gin Mandarine Musc by Bon Parfumeur is a Citrus Aromatic '
 'fragrance for women and men. 004 Gin Mandarine Musc wa

## Accords

In [15]:
retrieve_and_print_all_accords(driver)

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:13: DeprecationWarning: read_transaction has been renamed to execute_read
  accords = session.read_transaction(get_all_accords)


All Accords:
name: Leather, uri: http://www.example.org/perfumeKG#Accord_leather
name: warm spicy, uri: http://www.example.org/perfumeKG#Accord_warm-spicy
name: fresh spicy, uri: http://www.example.org/perfumeKG#Accord_fresh-spicy
name: Citrus, uri: http://www.example.org/perfumeKG#Accord_citrus
name: Aromatic, uri: http://www.example.org/perfumeKG#Accord_aromatic
name: Woody, uri: http://www.example.org/perfumeKG#Accord_woody
name: fresh, uri: http://www.example.org/perfumeKG#Accord_fresh
name: iris, uri: http://www.example.org/perfumeKG#Accord_iris
name: Fruity, uri: http://www.example.org/perfumeKG#Accord_fruity
name: Sweet, uri: http://www.example.org/perfumeKG#Accord_sweet
name: violet, uri: http://www.example.org/perfumeKG#Accord_violet
name: floral, uri: http://www.example.org/perfumeKG#Accord_floral
name: Powdery, uri: http://www.example.org/perfumeKG#َAccord_powdery
name: rose, uri: http://www.example.org/perfumeKG#Accord_rose
name: vanilla, uri: http://www.example.org/perfume

In [13]:
# selected_accord = {'name': 'coffee', 'uri': 'http://www.example.org/perfumeKG#coffee'}
# selected_accord = {'name': 'earthy', 'uri': 'http://www.example.org/perfumeKG#earthy'}
selected_accord = {'name': 'animalic', 'uri': 'http://www.example.org/perfumeKG#Accord_animalic'}

In [14]:
retrieve_perfumes_with_accord(driver, selected_accord['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:22: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:30: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:31: DeprecationWarning: read_transaction has been renamed to execute_read
  accords = session.read_transaction(get_perfumes)


All Perfumes with Searched Accord:
name: 06 black platinum, uri: http://www.example.org/perfumeKG#Perfume_06-black-platinum
name: 28°, uri: http://www.example.org/perfumeKG#Perfume_28
name: 66, uri: http://www.example.org/perfumeKG#Perfume_66
name: aesthete, uri: http://www.example.org/perfumeKG#Perfume_aesthete
name: aeternitas, uri: http://www.example.org/perfumeKG#Perfume_aeternitas
name: al fareed, uri: http://www.example.org/perfumeKG#Perfume_al-fareed
name: alien eau luminescente, uri: http://www.example.org/perfumeKG#Perfume_alien-eau-luminescente
name: alphaora 30, uri: http://www.example.org/perfumeKG#Perfume_alphaora-30
name: amalfi flowers, uri: http://www.example.org/perfumeKG#Perfume_amalfi-flowers
name: amber musc oil, uri: http://www.example.org/perfumeKG#Perfume_amber-musc-oil
name: amber oud ruby edition, uri: http://www.example.org/perfumeKG#Perfume_amber-oud-ruby-edition
name: amber oud ultra violet, uri: http://www.example.org/perfumeKG#Perfume_amber-oud-ultra-viole

In [16]:
selected_perfume = {'name': 'fairytale', 'uri': 'http://www.example.org/perfumeKG#Perfume_fairytale'}

In [17]:
retrieve_perfumes_similar_by_accord(driver, selected_perfume['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:40: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:48: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:49: DeprecationWarning: read_transaction has been renamed to execute_read
  accords = session.read_transaction(get_similar_perfumes)


Top 10 Similar Perfumes:
name: tuberose, uri: http://www.example.org/perfumeKG#Perfume_tuberose, score: 9
name: jardin exclusif, uri: http://www.example.org/perfumeKG#Perfume_jardin-exclusif, score: 8
name: tharwah gold, uri: http://www.example.org/perfumeKG#Perfume_tharwah-gold, score: 8
name: cinéma festival, uri: http://www.example.org/perfumeKG#Perfume_cinema-festival, score: 8
name: her golden secret, uri: http://www.example.org/perfumeKG#Perfume_her-golden-secret, score: 8
name: safanad, uri: http://www.example.org/perfumeKG#Perfume_safanad, score: 8
name: noir extreme, uri: http://www.example.org/perfumeKG#Perfume_noir-extreme, score: 8
name: ruby n vanilla neroli, uri: http://www.example.org/perfumeKG#Perfume_ruby-n-vanilla-neroli, score: 8
name: ambré, uri: http://www.example.org/perfumeKG#Perfume_ambre, score: 8
name: vain & naïve, uri: http://www.example.org/perfumeKG#Perfume_vain-naive, score: 8


## Notes

In [18]:
retrieve_and_print_all_notes(driver)

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:58: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:68: DeprecationWarning: read_transaction has been renamed to execute_read
  notes = session.read_transaction(get_all_notes)


All Notes:
name: Leather, uri: http://www.example.org/perfumeKG#Note_leather
name: Bergamot, uri: http://www.example.org/perfumeKG#Note_bergamot
name: patchouli, uri: http://www.example.org/perfumeKG#Note_patchouli
name: black pepper, uri: http://www.example.org/perfumeKG#Note_black-pepper
name: jasmine, uri: http://www.example.org/perfumeKG#Note_jasmine
name: coriander, uri: http://www.example.org/perfumeKG#Note_coriander
name: labdanum, uri: http://www.example.org/perfumeKG#Note_labdanum
name: muguet, uri: http://www.example.org/perfumeKG#Note_muguet
name: Sandalwood, uri: http://www.example.org/perfumeKG#Note_sandalwood
name: cassis, uri: http://www.example.org/perfumeKG#Note_cassis
name: rose, uri: http://www.example.org/perfumeKG#Note_rose
name: agarwood, uri: http://www.example.org/perfumeKG#Note_agarwood
name: clove, uri: http://www.example.org/perfumeKG#Note_clove
name: amber, uri: http://www.example.org/perfumeKG#Note_amber
name: rose centifola, uri: http://www.example.org/per

In [19]:
selected_note = {'name': 'vetiver', 'uri': 'http://www.example.org/perfumeKG#Note_vetiver'}

In [20]:
retrieve_perfumes_with_note(driver, selected_note['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:77: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:85: DeprecationWarning: read_transaction has been renamed to execute_read
  perfumes = session.read_transaction(get_perfumes)


All Perfumes with Searched Note:
name: 03 century eau de parfum, uri: http://www.example.org/perfumeKG#Perfume_03-century-eau-de-parfum
name: 08 - seylon eau de parfum, uri: http://www.example.org/perfumeKG#Perfume_08-seylon-eau-de-parfum
name: 1 million lucky, uri: http://www.example.org/perfumeKG#Perfume_1-million-lucky
name: 1 parfum extrait, uri: http://www.example.org/perfumeKG#Perfume_1-parfum-extrait
name: 10 am accord eau de parfum, uri: http://www.example.org/perfumeKG#Perfume_10-am-accord-eau-de-parfum
name: 10 corso como eau de parfum, uri: http://www.example.org/perfumeKG#Perfume_10-corso-como-eau-de-parfum
name: 154 cologne, uri: http://www.example.org/perfumeKG#Perfume_154-cologne
name: 1828 jules verne eau de parfum, uri: http://www.example.org/perfumeKG#Perfume_1828-jules-verne-eau-de-parfum
name: 1828 travel spray eau de parfum, uri: http://www.example.org/perfumeKG#Perfume_1828-travel-spray-eau-de-parfum
name: 1899 - ernest hemingway eau de parfum, uri: http://www.exa

In [21]:
selected_perfume_by_note = {'name': 'grace', 'uri': 'http://www.example.org/perfumeKG#Perfume_grace'}

In [22]:
retrieve_perfumes_similar_by_note(driver, selected_perfume_by_note['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:94: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:102: DeprecationWarning: read_transaction has been renamed to execute_read
  similar_perfumes = session.read_transaction(get_similar_perfumes)


Top 10 Similar Perfumes:
name: grace by grace coddington eau de toilette, uri: http://www.example.org/perfumeKG#Perfume_grace-by-grace-coddington-eau-de-toilette, score: 8
name: irth, uri: http://www.example.org/perfumeKG#Perfume_irth, score: 6
name: dancing light, uri: http://www.example.org/perfumeKG#Perfume_dancing-light, score: 6
name: moon light, uri: http://www.example.org/perfumeKG#Perfume_moon-light, score: 6
name: soleil d'italie eau de parfum, uri: http://www.example.org/perfumeKG#Perfume_soleil-d-italie-eau-de-parfum, score: 5
name: antidote, uri: http://www.example.org/perfumeKG#Perfume_antidote, score: 5
name: cashmere, uri: http://www.example.org/perfumeKG#Perfume_cashmere, score: 5
name: glamour just shine, uri: http://www.example.org/perfumeKG#Perfume_glamour-just-shine, score: 5
name: homem neo, uri: http://www.example.org/perfumeKG#Perfume_homem-neo, score: 5
name: augusto eau de parfum, uri: http://www.example.org/perfumeKG#Perfume_augusto-eau-de-parfum, score: 4


## Brands

In [3]:
retrieve_and_print_all_brands(driver)

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:113: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:120: DeprecationWarning: read_transaction has been renamed to execute_read
  brands = session.read_transaction(get_all_brands)


All Brands:
name: None, uri: http://www.example.org/perfumeKG#TUMI
name: ulrich lang, uri: http://www.example.org/perfumeKG#Brand_ulrichLang
name: histoires de parfums, uri: http://www.example.org/perfumeKG#Brand_histoiresDeParfums
name: chanel, uri: http://www.example.org/perfumeKG#Brand_chanel
name: rosendo mateu olfactive expressions, uri: http://www.example.org/perfumeKG#Brand_rosendoMateuOlfactiveExpressions
name: paco rabanne, uri: http://www.example.org/perfumeKG#Brand_pacoRabanne
name: yves saint laurent, uri: http://www.example.org/perfumeKG#Brand_yvesSaintLaurent
name: afnan, uri: http://www.example.org/perfumeKG#Brand_afnan
name: giorgio armani, uri: http://www.example.org/perfumeKG#Brand_giorgioArmani
name: bath & body works, uri: http://www.example.org/perfumeKG#Brand_bath&BodyWorks
name: yves rocher, uri: http://www.example.org/perfumeKG#Brand_yvesRocher
name: i profumi di firenze, uri: http://www.example.org/perfumeKG#Brand_iProfumiDiFirenze
name: the different company, 

In [3]:
selected_brand = {'name': 'zara', 'uri': 'http://www.example.org/perfumeKG#Brand_zara'}

In [5]:
print(pkg.getAccordsUsedByBrand(selected_brand['uri']))


        MATCH (brand:Brand {uri: 'http://www.example.org/perfumeKG#Brand_zara'})<-[:producedBy]-(perfume:Perfume)-[:hasAccord]->(accord:Accord)
        RETURN DISTINCT accord.uri AS accordUri, accord.label AS accordName
    


In [6]:
retrieve_notes_by_brand(driver, selected_brand['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:129: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:142: DeprecationWarning: read_transaction has been renamed to execute_read
  notes = session.read_transaction(get_notes)


Notes used by Brand URI: http://www.example.org/perfumeKG#Brand_zara
Note - name: orange blossom, uri: http://www.example.org/perfumeKG#Note_orangeBlossom
Note - name: bergamot, uri: http://www.example.org/perfumeKG#Note_bergamot
Note - name: mandarin orange, uri: http://www.example.org/perfumeKG#Note_mandarinOrange
Note - name: amber, uri: http://www.example.org/perfumeKG#Note_amber
Note - name: vanilla, uri: http://www.example.org/perfumeKG#Note_vanilla
Note - name: honey, uri: http://www.example.org/perfumeKG#Note_honey
Note - name: tobacco, uri: http://www.example.org/perfumeKG#Note_tobacco
Note - name: praline, uri: http://www.example.org/perfumeKG#Note_praline
Note - name: apple, uri: http://www.example.org/perfumeKG#Note_apple
Note - name: coconut water, uri: http://www.example.org/perfumeKG#Note_coconutWater
Note - name: rum, uri: http://www.example.org/perfumeKG#Note_rum
Note - name: guaiac wood, uri: http://www.example.org/perfumeKG#Note_guaiacWood
Note - name: black pepper, 

In [7]:
retrieve_accords_by_brand(driver, selected_brand['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:151: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:164: DeprecationWarning: read_transaction has been renamed to execute_read
  accords = session.read_transaction(get_accords)


Accords used by Brand URI: http://www.example.org/perfumeKG#Brand_zara
Accord - name: Accord_citrus, uri: http://www.example.org/perfumeKG#Accord_citrus
Accord - name: Accord_whiteFloral, uri: http://www.example.org/perfumeKG#Accord_whiteFloral
Accord - name: Accord_fruity, uri: http://www.example.org/perfumeKG#Accord_fruity
Accord - name: Accord_powdery, uri: http://www.example.org/perfumeKG#Accord_powdery
Accord - name: Accord_amber, uri: http://www.example.org/perfumeKG#Accord_amber
Accord - name: Accord_fresh, uri: http://www.example.org/perfumeKG#Accord_fresh
Accord - name: Accord_sweet, uri: http://www.example.org/perfumeKG#Accord_sweet
Accord - name: Accord_vanilla, uri: http://www.example.org/perfumeKG#Accord_vanilla
Accord - name: Accord_tobacco, uri: http://www.example.org/perfumeKG#Accord_tobacco
Accord - name: Accord_honey, uri: http://www.example.org/perfumeKG#Accord_honey
Accord - name: Accord_rum, uri: http://www.example.org/perfumeKG#Accord_rum
Accord - name: Accord_woo

In [8]:
print(pkg.getMostUsedAccordForBrand(selected_brand['uri']))


        MATCH (brand:Brand {uri: 'http://www.example.org/perfumeKG#Brand_zara'})<-[:producedBy]-(perfume:Perfume)-[:hasAccord]->(accord:Accord)
        WITH accord, COUNT(*) AS usageCount
        ORDER BY usageCount DESC
        RETURN accord.label AS accordName, accord.uri AS accordUri, usageCount
        LIMIT 1
    


In [7]:
selected_perfume = {'name': 'fairytale', 'uri': 'http://www.example.org/perfumeKG#Brand_royalCrown'}

In [5]:
pkg.get_most_similar_perfume_query(selected_perfume['uri'])

TypeError: get_most_similar_perfume_query() takes 0 positional arguments but 1 was given

In [6]:
print(pkg.perfumesShareAccords(selected_perfume['uri']))


        MATCH (specific:Perfume {uri: 'http://www.example.org/perfumeKG#fairytale'})-[:hasAccord]->(accord:Accord)<-[:hasAccord]-(other:Perfume)
        WHERE other <> specific
        RETURN other.label AS Perfume, other.uri AS uri, COUNT(accord) AS MutualAccords
        ORDER BY MutualAccords DESC
        LIMIT 10
    


## Perfume Similarity

In [2]:
selected_perfume = {'name': 'fairytale', 'uri': 'http://www.example.org/perfumeKG#Perfume_fairytale'}

print(pkg.getAccordSimilarityScore(selected_perfume['uri']))


        MATCH (specific:Perfume {uri: 'http://www.example.org/perfumeKG#Perfume_fairytale'})
        OPTIONAL MATCH (specific)-[:hasAccord]->(specificAccord:Accord)
        WITH specific, COLLECT(DISTINCT specificAccord) AS specificAccords

        MATCH (other:Perfume)
        WHERE other <> specific
        OPTIONAL MATCH (other)-[:hasAccord]->(otherAccord:Accord)
        WITH specific, specificAccords, other, COLLECT(DISTINCT otherAccord) AS otherAccords

        WITH other,
            [accord IN specificAccords WHERE accord IN otherAccords] AS mutualAccords,
            [accord IN specificAccords WHERE NOT accord IN otherAccords] +
            [accord IN otherAccords WHERE NOT accord IN specificAccords] AS notMutualAccords,
            SIZE([accord IN specificAccords WHERE accord IN otherAccords]) AS MutualAccordsCount,
            SIZE([accord IN specificAccords WHERE NOT accord IN otherAccords] +
                [accord IN otherAccords WHERE NOT accord IN specificAccords]) AS N

In [8]:
retrieve_combined_similarity_scores(driver, selected_perfume['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:198: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5556\797122043.py:209: DeprecationWarning: read_transaction has been renamed to execute_read
  perfumes_info = session.read_transaction(get_similarity_info)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n        MATCH (specific:Perfume {uri: 'http://www.example.org/perfumeKG#Perfume_fairytale'})\n        OPTIONAL MATCH (specific)-[:hasAccord]->(specificAccord:Accord)\n        WITH specific, COLLECT(DISTINCT specificAccord) AS specificAccords\n\n     

Top 10 Combined Similarity Scores:
Name: galloway
URI: http://www.example.org/perfumeKG#Perfume_galloway
Accord Similarity: 0.5
Note Similarity: 0.2222222222222222
Average Similarity: 0.3611111111111111
---
Name: cherry of cashmere
URI: http://www.example.org/perfumeKG#Perfume_cherryOfCashmere
Accord Similarity: 0.5
Note Similarity: 0.2
Average Similarity: 0.35
---
Name: classique la robe chinoise
URI: http://www.example.org/perfumeKG#Perfume_classiqueLaRobeChinoise
Accord Similarity: 0.5
Note Similarity: 0.2
Average Similarity: 0.35
---
Name: white soul gold & diamonds
URI: http://www.example.org/perfumeKG#Perfume_whiteSoulGold&Diamonds
Accord Similarity: 0.5
Note Similarity: 0.2
Average Similarity: 0.35
---
Name: sweet pulp
URI: http://www.example.org/perfumeKG#Perfume_sweetPulp
Accord Similarity: 0.6363636363636364
Note Similarity: 0.058823529411764705
Average Similarity: 0.34759358288770054
---
Name: day dreams
URI: http://www.example.org/perfumeKG#Perfume_dayDreams
Accord Similari